<a href="https://colab.research.google.com/github/alelom/Notebooks/blob/master/Statistical%20Learning/Pytorch/Exercise%20Files/Using_Optimizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using optimizers

In [1]:
# Setting seeds to try and ensure we have the same results - this is not guaranteed across PyTorch releases.
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import numpy as np
np.random.seed(0)

In [2]:
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import nn

mean, std = (0.5,), (0.5,)

# Create a transform and normalise data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean, std)
                              ])

# Download FMNIST training dataset and load training data
trainset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download FMNIST test dataset and load test data
testset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Extracting /root/.pytorch/FMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/FMNIST/FashionMNIST/raw


Extracting /root/.pytorch/FMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/FMNIST/FashionMNIST/raw


Extracting /root/.pytorch/FMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/FMNIST/FashionMNIST/raw


Extracting /root/.pytorch/FMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/FMNIST/FashionMNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [3]:
class FMNIST(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784, 128)
    self.fc2 = nn.Linear(128,64)
    self.fc3 = nn.Linear(64,10)
    
  def forward(self, x):
    x = x.view(x.shape[0], -1)
    
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    x = F.log_softmax(x, dim=1)
    
    return x
    
#model = FMNIST()   

In [4]:
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

In [5]:
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

In [7]:
from torch import optim

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01) # lr = Learning Rate


In [8]:
output = model(images)
loss = criterion(output, labels)
loss.backward()
print('Initial weights : ',model[0].weight)
print('Initial weights gradient : ',model[0].weight.grad)
        

Initial weights :  Parameter containing:
tensor([[-0.0003,  0.0192, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0150, -0.0104,  ..., -0.0203, -0.0060, -0.0299],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0018, -0.0295,  0.0085,  ..., -0.0037,  0.0036,  0.0300],
        [-0.0233, -0.0220, -0.0064,  ...,  0.0115, -0.0324, -0.0158],
        [ 0.0309,  0.0066,  0.0125,  ...,  0.0286,  0.0350, -0.0105]],
       requires_grad=True)
Initial weights gradient :  tensor([[-0.0004, -0.0004, -0.0004,  ..., -0.0007, -0.0006, -0.0004],
        [ 0.0069,  0.0069,  0.0069,  ...,  0.0072,  0.0070,  0.0069],
        [-0.0015, -0.0015, -0.0015,  ..., -0.0016, -0.0015, -0.0015],
        ...,
        [ 0.0018,  0.0018,  0.0018,  ...,  0.0017,  0.0017,  0.0018],
        [ 0.0019,  0.0019,  0.0019,  ...,  0.0019,  0.0019,  0.0019],
        [ 0.0017,  0.0017,  0.0017,  ...,  0.0016,  0.0017,  0.0017]])


In [9]:
# Update the weights using the step() function of the optimiser
optimizer.step()

In [10]:
# The weights have been updated *ever so slightly*. Some minor differences are visible.
print('Initial weights : ',model[0].weight)
print('Initial weights gradient : ',model[0].weight.grad)

Initial weights :  Parameter containing:
tensor([[-0.0003,  0.0192, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0151, -0.0105,  ..., -0.0203, -0.0060, -0.0300],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0018, -0.0296,  0.0085,  ..., -0.0037,  0.0036,  0.0300],
        [-0.0233, -0.0221, -0.0064,  ...,  0.0115, -0.0324, -0.0158],
        [ 0.0309,  0.0066,  0.0125,  ...,  0.0285,  0.0350, -0.0105]],
       requires_grad=True)
Initial weights gradient :  tensor([[-0.0004, -0.0004, -0.0004,  ..., -0.0007, -0.0006, -0.0004],
        [ 0.0069,  0.0069,  0.0069,  ...,  0.0072,  0.0070,  0.0069],
        [-0.0015, -0.0015, -0.0015,  ..., -0.0016, -0.0015, -0.0015],
        ...,
        [ 0.0018,  0.0018,  0.0018,  ...,  0.0017,  0.0017,  0.0018],
        [ 0.0019,  0.0019,  0.0019,  ...,  0.0019,  0.0019,  0.0019],
        [ 0.0017,  0.0017,  0.0017,  ...,  0.0016,  0.0017,  0.0017]])


In [12]:
# Now that we have updated the weights, we don't want to accumulate the gradients.
# We need to zero the gradients.
optimizer.zero_grad()

In [13]:
# Printing again we see that we have the updated weights,
# but all the gradients have been zeroed out.
print('Initial weights : ',model[0].weight)
print('Initial weights gradient : ',model[0].weight.grad)

Initial weights :  Parameter containing:
tensor([[-0.0003,  0.0192, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0151, -0.0105,  ..., -0.0203, -0.0060, -0.0300],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0018, -0.0296,  0.0085,  ..., -0.0037,  0.0036,  0.0300],
        [-0.0233, -0.0221, -0.0064,  ...,  0.0115, -0.0324, -0.0158],
        [ 0.0309,  0.0066,  0.0125,  ...,  0.0285,  0.0350, -0.0105]],
       requires_grad=True)
Initial weights gradient :  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [18]:
# Now we want to do a loop.
model = FMNIST()
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 1 # let's see what happens for 1 epoch first.

for i in range(num_epochs):
    cum_loss = 0
    image_batchIndex = 0

    for image_batchIndex, (images, labels) in enumerate(trainloader, 1): # the 1 means that enumerate will start counting from 1. The multi-output of trainloader needs to be wrapped in a tuple to be identified.
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        cum_loss += loss.item()
        print(f"Index of the image in this batch: {image_batchIndex}, Loss: {loss.item()}")
       
     
    print(f"Training loss: {cum_loss/len(trainloader)}")

Index of the image in this batch: 1, Loss: 2.307603359222412
Index of the image in this batch: 2, Loss: 2.3155648708343506
Index of the image in this batch: 3, Loss: 2.3367562294006348
Index of the image in this batch: 4, Loss: 2.3264882564544678
Index of the image in this batch: 5, Loss: 2.3059499263763428
Index of the image in this batch: 6, Loss: 2.3098182678222656
Index of the image in this batch: 7, Loss: 2.3145508766174316
Index of the image in this batch: 8, Loss: 2.2938530445098877
Index of the image in this batch: 9, Loss: 2.3016247749328613
Index of the image in this batch: 10, Loss: 2.297064781188965
Index of the image in this batch: 11, Loss: 2.3233449459075928
Index of the image in this batch: 12, Loss: 2.2991514205932617
Index of the image in this batch: 13, Loss: 2.3026156425476074
Index of the image in this batch: 14, Loss: 2.2786035537719727
Index of the image in this batch: 15, Loss: 2.284822940826416
Index of the image in this batch: 16, Loss: 2.2843384742736816
Inde

In [16]:
# 938 images comes from:
# Train data set = 60000 images.
# We divided it in 64 batches.
60000 / 64

937.5